In [1]:
from scrapy.item import Field
from scrapy.item import Item
from scrapy.spiders import CrawlSpider, Rule
from scrapy.spiders import Spider
from scrapy.selector import Selector
from scrapy.linkextractors import LinkExtractor
from scrapy.loader import ItemLoader
from scrapy.crawler import CrawlerProcess

import pandas as pd

import urllib.parse

import re

In [2]:
#Definimos la clase Pokemon que nos servirá para ir extrayendo y almacenando los datos de scrapy
class Pokemon(Item):
    nombre = Field()
    tipos = Field()
    generacion = Field()
    categoria = Field()
    habilidades = Field()
    peso = Field()
    altura = Field()
    ps = Field()
    ataque = Field()
    defensa = Field()
    ataque_s = Field()
    defensa_s = Field()
    velocidad = Field()
    foto = Field()

In [3]:
#Lista de pokemones
df_pokemon = pd.read_csv('pokedex.csv')
df_pokemon

,Titulo,Dominio
0,1 - Bulbasaur,/el/pokedex/bulbasaur
1,2 - Ivysaur,/el/pokedex/ivysaur
2,3 - Venusaur,/el/pokedex/venusaur
3,4 - Charmander,/el/pokedex/charmander
4,5 - Charmeleon,/el/pokedex/charmeleon
...,...,...
1020,"1,021 - Electrofuria",/el/pokedex/electrofuria
1021,"1,022 - Ferromole",/el/pokedex/ferromole
1022,"1,023 - Ferrotesta",/el/pokedex/ferrotesta
1023,"1,024 - Terapagos",/el/pokedex/terapagos


In [4]:
import urllib.parse
urls = []
#Recorremos cada fila del daataframe para ir construyendo la url que consultaremos para cada pokemon en la página Wikidex
for index, row in df_pokemon.iterrows():
    nombre = row['Titulo'].split('-', 1)[1].replace('\n', '').replace('\t', '').strip() #
    nombre = nombre.replace('’', "'")
    # Convertir a ASCII URL encoding
    ref = urllib.parse.quote(nombre).replace('%20', '_') #
    #Construimos las url añadiendo el nombre del pokemon
    url = 'https://www.wikidex.net/wiki/' + ref
    urls.append(url)

In [5]:
start_urls = urls
start_urls

['https://www.wikidex.net/wiki/Bulbasaur',
 'https://www.wikidex.net/wiki/Ivysaur',
 'https://www.wikidex.net/wiki/Venusaur',
 'https://www.wikidex.net/wiki/Charmander',
 'https://www.wikidex.net/wiki/Charmeleon',
 'https://www.wikidex.net/wiki/Charizard',
 'https://www.wikidex.net/wiki/Squirtle',
 'https://www.wikidex.net/wiki/Wartortle',
 'https://www.wikidex.net/wiki/Blastoise',
 'https://www.wikidex.net/wiki/Caterpie',
 'https://www.wikidex.net/wiki/Metapod',
 'https://www.wikidex.net/wiki/Butterfree',
 'https://www.wikidex.net/wiki/Weedle',
 'https://www.wikidex.net/wiki/Kakuna',
 'https://www.wikidex.net/wiki/Beedrill',
 'https://www.wikidex.net/wiki/Pidgey',
 'https://www.wikidex.net/wiki/Pidgeotto',
 'https://www.wikidex.net/wiki/Pidgeot',
 'https://www.wikidex.net/wiki/Rattata',
 'https://www.wikidex.net/wiki/Raticate',
 'https://www.wikidex.net/wiki/Spearow',
 'https://www.wikidex.net/wiki/Fearow',
 'https://www.wikidex.net/wiki/Ekans',
 'https://www.wikidex.net/wiki/Arbok',


In [6]:
# CLASE CORE - Al querer hacer extraccion de multiples paginas, heredamos de CrawlSpider
class PokemonPokedex(CrawlSpider):
    name = 'pokemon'
    custom_settings = {
        'USER_AGENT': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36',
        'FEED_EXPORT_ENCODING': 'utf-8', # Para evitar problemas con codificacion de simbolos
        'FEED_EXPORT_FIELDS': ['nombre', 'tipos', 'generacion', 'categoria','habilidades','peso','altura','ps','ataque','defensa','ataque_s','defensa_s','velocidad','foto']
    }

    # Url semillas
    #Vamos recorriendo la web informativa de cada pokemon
    start_urls = start_urls
    download_delay = 1.5

    def parse_start_url(self, response):
        sel = Selector(response)
        item = ItemLoader(Pokemon(),sel)
        
        #Obtener el nombre
        nombre = sel.xpath("//div[@id='nombrepokemon']/text()").get()
        item.add_value('nombre',nombre)
        
        #Obtener los tipos
        tipos = []
        tipos_tittle = sel.xpath('//tr[@title="Tipos a los que pertenece"]/td/a[contains(@title,"Tipo")]/@title').getall()
        for tipo in tipos_tittle:
            tipos.append(tipo)
        item.add_value('tipos',tipos)
        
        #Obtener la generacion
        generacion = sel.xpath("//tr[@title='Generación en la que apareció por primera vez']/td/a[contains(@title,'generación')][1]/@title").get()
        item.add_value('generacion',generacion)
        
        #Obtener la categoria
        categoria = sel.xpath("//tr[@title='Categoría']/td/text()").get()
        item.add_value('categoria',categoria.replace('\n', '').replace('\t', '').strip())
        
        #Obtener habilidades
        habilidades = []
        habilidades_title = sel.xpath("//tr[@title='Habilidades que puede conocer']/td/a/@title").getall()
        for habilidad in habilidades_title:
            habilidades.append(habilidad)
        item.add_value('habilidades',habilidades)
        
        #Obtener peso
        peso = sel.xpath("//tr[contains(@title,'Peso del')]/td/text()").get()
        item.add_value('peso',peso.replace('\n', '').replace('\t', '').strip())
        
        #Obtener altura 
        altura = sel.xpath("//tr[contains(@title,'Altura del')]/td/text()").get()
        item.add_value('altura',altura.replace('\n', '').replace('\t', '').strip())        
        
        #Obteniendo los stats
        contenedor_stats = sel.xpath("//th[@class='enlacesblancos']/following-sibling::*[1]")
        contador = 0
        for stat in contenedor_stats:
            score = stat.xpath("./text()").get().replace('\n', '').replace('\t', '').strip()
            if contador == 0:
                ps = score
            elif contador == 1:
                ataque = score
            elif contador == 2:
                defensa = score
            elif contador == 3:
                ataque_s = score
            elif contador == 4:
                defensa_s = score
            elif contador == 5:
                velocidad = score
            else:
                break
            contador += 1

        #Obtener url de la foto del pokemon
        foto = sel.xpath("//div[@class='vnav_datos']//img[contains(@alt,'Ilustración de')]/@src").get()
        item.add_value('foto',foto.replace('\n', '').replace('\t', '').strip()) 
        
        
        item.add_value('ps',ps)
        item.add_value('ataque',ataque)
        item.add_value('defensa',defensa)
        item.add_value('ataque_s',ataque_s)
        item.add_value('defensa_s',defensa_s)
        item.add_value('velocidad',velocidad)
        item.add_value('foto',foto)

        yield item.load_item()

In [7]:
#Inciamos el proceso de scrapy y almacenamos los datos scrapeados en un csv
process = CrawlerProcess( settings= {'FEEDS': {'pokedex_detalle.csv': {'format':'csv'}}} )
process.crawl(PokemonPokedex)
process.start()

2024-08-07 23:44:15 [scrapy.utils.log] INFO: Scrapy 2.11.2 started (bot: scrapybot)
2024-08-07 23:44:15 [scrapy.utils.log] INFO: Versions: lxml 4.9.3.0, libxml2 2.10.4, cssselect 1.2.0, parsel 1.9.1, w3lib 1.21.0, Twisted 24.3.0, Python 3.11.5 | packaged by Anaconda, Inc. | (main, Sep 11 2023, 13:26:23) [MSC v.1916 64 bit (AMD64)], pyOpenSSL 23.2.0 (OpenSSL 3.0.12 24 Oct 2023), cryptography 41.0.3, Platform Windows-10-10.0.18362-SP0
2024-08-07 23:44:15 [scrapy.addons] INFO: Enabled addons:
[]
2024-08-07 23:44:15 [py.warnings] WARNING: D:\Programas\Anaconda\Lib\site-packages\scrapy\utils\request.py:254: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting.

It is also the default value. In other words, it is normal to get this warning if you have not defined a value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting. This is so for backward compatibility reasons, but it will change in a future version of Scrapy.

See the docume

2024-08-07 23:44:28 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.wikidex.net/wiki/Squirtle> (referer: None)
2024-08-07 23:44:28 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.wikidex.net/wiki/Squirtle>
{'altura': ['0,5 m'],
 'ataque': ['48'],
 'ataque_s': ['50'],
 'categoria': ['Tortuguita'],
 'defensa': ['65'],
 'defensa_s': ['64'],
 'foto': ['https://images.wikidexcdn.net/mwuploads/wikidex/thumb/e/e3/latest/20160309230820/Squirtle.png/200px-Squirtle.png',
          'https://images.wikidexcdn.net/mwuploads/wikidex/thumb/e/e3/latest/20160309230820/Squirtle.png/200px-Squirtle.png'],
 'generacion': ['Primera generación'],
 'habilidades': ['Torrente'],
 'nombre': ['Squirtle'],
 'peso': ['9,0 kg'],
 'ps': ['44'],
 'tipos': ['Tipo agua'],
 'velocidad': ['43']}
2024-08-07 23:44:30 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.wikidex.net/wiki/Wartortle> (referer: None)
2024-08-07 23:44:30 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.wikide

2024-08-07 23:44:47 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.wikidex.net/wiki/Pidgeot> (referer: None)
2024-08-07 23:44:48 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.wikidex.net/wiki/Pidgeot>
{'altura': ['1,5 m'],
 'ataque': ['80'],
 'ataque_s': ['70'],
 'categoria': ['Pájaro'],
 'defensa': ['75'],
 'defensa_s': ['70'],
 'foto': ['https://images.wikidexcdn.net/mwuploads/wikidex/thumb/a/a9/latest/20141214190416/Pidgeot.png/200px-Pidgeot.png',
          'https://images.wikidexcdn.net/mwuploads/wikidex/thumb/a/a9/latest/20141214190416/Pidgeot.png/200px-Pidgeot.png'],
 'generacion': ['Primera generación'],
 'habilidades': ['Vista lince', 'Tumbos'],
 'nombre': ['Pidgeot'],
 'peso': ['39,5 kg'],
 'ps': ['83'],
 'tipos': ['Tipo normal', 'Tipo volador'],
 'velocidad': ['91']}
2024-08-07 23:44:50 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.wikidex.net/wiki/Rattata> (referer: None)
2024-08-07 23:44:50 [scrapy.core.scraper] DEBUG: Scraped from <200

2024-08-07 23:45:08 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.wikidex.net/wiki/Nidoran%E2%99%80> (referer: None)
2024-08-07 23:45:08 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.wikidex.net/wiki/Nidoran%E2%99%80>
{'altura': ['0,4 m'],
 'ataque': ['47'],
 'ataque_s': ['40'],
 'categoria': ['Pin veneno'],
 'defensa': ['52'],
 'defensa_s': ['40'],
 'foto': ['https://images.wikidexcdn.net/mwuploads/wikidex/thumb/c/c8/latest/20230613021248/Nidoran%E2%99%80.png/200px-Nidoran%E2%99%80.png',
          'https://images.wikidexcdn.net/mwuploads/wikidex/thumb/c/c8/latest/20230613021248/Nidoran%E2%99%80.png/200px-Nidoran%E2%99%80.png'],
 'generacion': ['Primera generación'],
 'habilidades': ['Punto tóxico', 'Rivalidad'],
 'nombre': ['Nidoran♀'],
 'peso': ['7,0 kg'],
 'ps': ['55'],
 'tipos': ['Tipo veneno'],
 'velocidad': ['41']}
2024-08-07 23:45:11 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.wikidex.net/wiki/Nidorina> (referer: None)
2024-08-07 23:45:11

2024-08-07 23:45:26 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.wikidex.net/wiki/Jigglypuff> (referer: None)
2024-08-07 23:45:27 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.wikidex.net/wiki/Jigglypuff>
{'altura': ['0,5 m'],
 'ataque': ['45'],
 'ataque_s': ['45'],
 'categoria': ['Globo'],
 'defensa': ['20'],
 'defensa_s': ['25'],
 'foto': ['https://images.wikidexcdn.net/mwuploads/wikidex/thumb/a/af/latest/20150110232910/Jigglypuff.png/200px-Jigglypuff.png',
          'https://images.wikidexcdn.net/mwuploads/wikidex/thumb/a/af/latest/20150110232910/Jigglypuff.png/200px-Jigglypuff.png'],
 'generacion': ['Primera generación'],
 'habilidades': ['Gran encanto', 'Tenacidad'],
 'nombre': ['Jigglypuff'],
 'peso': ['5,5 kg'],
 'ps': ['115'],
 'tipos': ['Tipo normal', 'Tipo hada'],
 'velocidad': ['20']}
2024-08-07 23:45:28 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.wikidex.net/wiki/Wigglytuff> (referer: None)
2024-08-07 23:45:28 [scrapy.core.scraper] 

...

2024-08-08 00:14:09 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.wikidex.net/wiki/Clodsire>
{'altura': ['1,8 m'],
 'ataque': ['75'],
 'ataque_s': ['45'],
 'categoria': ['Pez Púas'],
 'defensa': ['60'],
 'defensa_s': ['100'],
 'foto': ['https://images.wikidexcdn.net/mwuploads/wikidex/thumb/1/1b/latest/20230113072717/Clodsire.png/200px-Clodsire.png',
          'https://images.wikidexcdn.net/mwuploads/wikidex/thumb/1/1b/latest/20230113072717/Clodsire.png/200px-Clodsire.png'],
 'generacion': ['Novena generación'],
 'habilidades': ['Punto tóxico', 'Absorbe agua'],
 'nombre': ['Clodsire'],
 'peso': ['223,0 kg'],
 'ps': ['130'],
 'tipos': ['Tipo veneno', 'Tipo tierra'],
 'velocidad': ['20']}
2024-08-08 00:14:10 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.wikidex.net/wiki/Farigiraf> (referer: None)
2024-08-08 00:14:10 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.wikidex.net/wiki/Farigiraf>
{'altura': ['3,2 m'],
 'ataque': ['90'],
 'ataque_s': ['110'],

2024-08-08 00:14:27 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.wikidex.net/wiki/Ferrodada> (referer: None)
2024-08-08 00:14:27 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.wikidex.net/wiki/Ferrodada>
{'altura': ['0,9 m'],
 'ataque': ['112'],
 'ataque_s': ['72'],
 'categoria': ['Paradoja'],
 'defensa': ['120'],
 'defensa_s': ['70'],
 'foto': ['https://images.wikidexcdn.net/mwuploads/wikidex/thumb/1/17/latest/20230112194612/Ferrodada.png/200px-Ferrodada.png',
          'https://images.wikidexcdn.net/mwuploads/wikidex/thumb/1/17/latest/20230112194612/Ferrodada.png/200px-Ferrodada.png'],
 'generacion': ['Novena generación'],
 'habilidades': ['Carga cuark'],
 'nombre': ['Ferrodada'],
 'peso': ['240,0 kg'],
 'ps': ['90'],
 'tipos': ['Tipo tierra', 'Tipo acero'],
 'velocidad': ['106']}
2024-08-08 00:14:30 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.wikidex.net/wiki/Ferrosaco> (referer: None)
2024-08-08 00:14:30 [scrapy.core.scraper] DEBUG: Scraped 

2024-08-08 00:14:45 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.wikidex.net/wiki/Gholdengo> (referer: None)
2024-08-08 00:14:45 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.wikidex.net/wiki/Gholdengo>
{'altura': ['1,2 m'],
 'ataque': ['60'],
 'ataque_s': ['133'],
 'categoria': ['Tesoro'],
 'defensa': ['95'],
 'defensa_s': ['91'],
 'foto': ['https://images.wikidexcdn.net/mwuploads/wikidex/thumb/6/65/latest/20230112200229/Gholdengo.png/200px-Gholdengo.png',
          'https://images.wikidexcdn.net/mwuploads/wikidex/thumb/6/65/latest/20230112200229/Gholdengo.png/200px-Gholdengo.png'],
 'generacion': ['Novena generación'],
 'habilidades': ['Cuerpo áureo'],
 'nombre': ['Gholdengo'],
 'peso': ['30,0 kg'],
 'ps': ['87'],
 'tipos': ['Tipo acero', 'Tipo fantasma'],
 'velocidad': ['84']}
2024-08-08 00:14:47 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.wikidex.net/wiki/Wo-Chien> (referer: None)
2024-08-08 00:14:47 [scrapy.core.scraper] DEBUG: Scraped fro

2024-08-08 00:15:03 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.wikidex.net/wiki/Ferroverdor> (referer: None)
2024-08-08 00:15:03 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.wikidex.net/wiki/Ferroverdor>
{'altura': ['1,5 m'],
 'ataque': ['130'],
 'ataque_s': ['70'],
 'categoria': ['Paradoja'],
 'defensa': ['88'],
 'defensa_s': ['108'],
 'foto': ['https://images.wikidexcdn.net/mwuploads/wikidex/thumb/f/ff/latest/20230227145227/Ferroverdor.png/200px-Ferroverdor.png',
          'https://images.wikidexcdn.net/mwuploads/wikidex/thumb/f/ff/latest/20230227145227/Ferroverdor.png/200px-Ferroverdor.png'],
 'generacion': ['Novena generación'],
 'habilidades': ['Carga cuark'],
 'nombre': ['Ferroverdor'],
 'peso': ['125,0 kg'],
 'ps': ['90'],
 'tipos': ['Tipo planta', 'Tipo psíquico'],
 'velocidad': ['104']}
2024-08-08 00:15:05 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.wikidex.net/wiki/Dipplin> (referer: None)
2024-08-08 00:15:05 [scrapy.core.scraper] 

2024-08-08 00:15:19 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.wikidex.net/wiki/Flamariete> (referer: None)
2024-08-08 00:15:20 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.wikidex.net/wiki/Flamariete>
{'altura': ['3,5 m'],
 'ataque': ['115'],
 'ataque_s': ['65'],
 'categoria': ['Paradoja'],
 'defensa': ['121'],
 'defensa_s': ['93'],
 'foto': ['https://images.wikidexcdn.net/mwuploads/wikidex/thumb/c/c0/latest/20240125162527/Flamariete.png/200px-Flamariete.png',
          'https://images.wikidexcdn.net/mwuploads/wikidex/thumb/c/c0/latest/20240125162527/Flamariete.png/200px-Flamariete.png'],
 'generacion': ['Novena generación'],
 'habilidades': ['Paleosíntesis'],
 'nombre': ['Flamariete'],
 'peso': ['590,0 kg'],
 'ps': ['105'],
 'tipos': ['Tipo fuego', 'Tipo dragón'],
 'velocidad': ['91']}
2024-08-08 00:15:21 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.wikidex.net/wiki/Electrofuria> (referer: None)
2024-08-08 00:15:21 [scrapy.core.scraper] DEB

In [8]:
#Cargamos en un dataframe la daata del csv
df_probando = pd.read_csv('pokedex_detalle.csv')
df_probando

,nombre,tipos,generacion,categoria,habilidades,peso,altura,ps,ataque,defensa,ataque_s,defensa_s,velocidad,foto
0,Bulbasaur,"Tipo planta,Tipo veneno",Primera generación,Semilla,Espesura,"6,9 kg","0,7 m",45,49,49,65,65,45,https://images.wikidexcdn.net/mwuploads/wikide...
1,Ivysaur,"Tipo planta,Tipo veneno",Primera generación,Semilla,Espesura,"13,0 kg","1,0 m",60,62,63,80,80,60,https://images.wikidexcdn.net/mwuploads/wikide...
2,Venusaur,"Tipo planta,Tipo veneno",Primera generación,Semilla,Espesura,"100,0 kg","2,0 m",80,82,83,100,100,80,https://images.wikidexcdn.net/mwuploads/wikide...
3,Charmander,Tipo fuego,Primera generación,Lagartija,Mar llamas,"8,5 kg","0,6 m",39,52,43,60,50,65,https://images.wikidexcdn.net/mwuploads/wikide...
4,Charmeleon,Tipo fuego,Primera generación,Llama,Mar llamas,"19,0 kg","1,1 m",58,64,58,80,65,80,https://images.wikidexcdn.net/mwuploads/wikide...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1020,Electrofuria,"Tipo eléctrico,Tipo dragón",Novena generación,Paradoja,Paleosíntesis,"480,0 kg","5,2 m",125,73,91,137,89,75,https://images.wikidexcdn.net/mwuploads/wikide...
1021,Ferromole,"Tipo roca,Tipo psíquico",Novena generación,Paradoja,Carga cuark,"162,5 kg","1,5 m",90,120,80,68,108,124,https://images.wikidexcdn.net/mwuploads/wikide...
1022,Ferrotesta,"Tipo acero,Tipo psíquico",Novena generación,Paradoja,Carga cuark,"156,0 kg","1,6 m",90,72,100,122,108,98,https://images.wikidexcdn.net/mwuploads/wikide...
1023,Terapagos,Tipo normal,Novena generación,Teracristal,Teracambio,"6,5 kg","0,2 m",90,65,85,65,85,60,https://images.wikidexcdn.net/mwuploads/wikide...
